In [22]:
import numpy as np
import sympy as sym

In [23]:
n = 20
x = sym.Symbol('x', real = True)
y = sym.Symbol('y',real=True)

In [24]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [25]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [26]:
def GetLaguerre(n, x):
    if n == 0:
        poly = sym.Number(1)
    elif n==1:
        poly = 1-x
    else:
        poly = ((2*(n-1)+1-x)*GetLaguerre(n-1,x)-(n-1)*GetLaguerre(n-2,x))/n
    return sym.expand(poly,x)

In [27]:
def GetDLaguerre(n,x):
    Pn = GetLaguerre(n,x)
    return sym.diff(Pn,x,1)

In [28]:
def GetAllRootsGLag (n):
    xn = np.linspace(0,n+(n-1)*np.sqrt(n),100)
    
    Laguerre = []
    DLaguerre = []
    
    for i in range(n+1):
        Laguerre.append(GetLaguerre(i,x))
        DLaguerre.append(GetDLaguerre(i,x))
    
    poly = sym.lambdify([x],Laguerre[n],'numpy')
    Dpoly = sym.lambdify([x],DLaguerre[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [29]:
def GetWeightsGLag(n):

    Roots = GetAllRootsGLag(n)

    Laguerre = []
    
    for i in range(n+2):
        Laguerre.append(GetLaguerre(i,x))
    
    poly = sym.lambdify([x],Laguerre[n+1],'numpy')
    Weights = Roots/((n+1)**2*poly(Roots)**2)
    
    return Weights

In [30]:
Roots,Weights = np.polynomial.laguerre.laggauss(n)
pesos = GetWeightsGLag(n)
raices = GetAllRootsGLag(n)
print(pesos,raices)
print(Weights)
print("----------------------------------------------------------------------")
print(Roots)

[0.1687468  0.29125436 0.2666861  0.00620255] [0.07053989 0.37212682 0.9165821  5.61517497]
[1.68746802e-01 2.91254362e-01 2.66686103e-01 1.66002453e-01
 7.48260647e-02 2.49644173e-02 6.20255084e-03 1.14496239e-03
 1.55741773e-04 1.54014409e-05 1.08648637e-06 5.33012091e-08
 1.75798118e-09 3.72550240e-11 4.76752925e-13 3.37284424e-15
 1.15501434e-17 1.53952214e-20 5.28644273e-24 1.65645661e-28]
----------------------------------------------------------------------
[ 0.07053989  0.37212682  0.9165821   1.70730653  2.74919926  4.04892531
  5.61517497  7.45901745  9.59439287 12.03880255 14.81429344 17.94889552
 21.47878824 25.45170279 29.93255463 35.01343424 40.83305706 47.61999405
 55.81079575 66.52441653]
